In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from skimage.io import imread
from sklearn.model_selection import train_test_split

In [2]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras import Sequential
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.layers import Dense,Conv2D,Flatten

In [3]:
df = pd.read_csv('../input/brain-tumor/Brain Tumor.csv')

In [4]:
df.head()

In [5]:
dfmain = df[['Image', 'Class']].copy()

In [6]:
dfmain.head()

In [7]:
dfmain['Class'].value_counts()

In [8]:
dfmain.shape

In [9]:
sns.countplot(dfmain['Class'])

In [10]:
path_list = []
base_path = '../input/brain-tumor/Brain Tumor/Brain Tumor'
for entry in os.listdir(base_path):
    path_list.append( os.path.join(base_path,entry))

In [11]:
pathes_dict={os.path.splitext(os.path.basename(x))[0]: x for x in path_list}
dfmain['pathes'] = dfmain['Image'].map(pathes_dict.get)

In [12]:
dfmain.head()

In [13]:
dfmain['pathes']

In [14]:
for x in range (0 ,9):
    img = imread(dfmain['pathes'][x])
    plt.imshow(img)
    plt.subplot(3,3,x+1)

In [15]:
from PIL.Image import open
dfmain['pixels']=dfmain['pathes'].map(lambda x:np.asarray(open(x).resize((224,224))))

In [16]:
dfmain

In [17]:
dfmain['pixels']

In [18]:
image_list = []
for i in range(0,len(dfmain)):
    # load image
    brain_img = dfmain['pixels'][i].astype(np.float32)
    img_array = image.img_to_array(brain_img)
     # append to list of all images
    image_list.append(preprocess_input(img_array))
    
# convert image list to single array
# Our feature
X = np.array(image_list)

# print shape of X
print(X.shape)

In [19]:
y = np.array(dfmain.Class)

In [20]:
y

In [21]:
X_train,X_test,y_train,y_test = train_test_split(X,y
                                                 ,test_size=0.2
                                                ,random_state=42)


print('The shape of the X_train :'+' '+str(X_train.shape))
print('The size of the X_train :'+' '+str(X_train.shape[0]))
print('The shape of the X_test :'+' '+str(X_test.shape))
print('The size of the X_test:'+' '+str(X_test.shape[0]))

In [22]:
import tensorflow as tf

In [23]:
num_classes = 1
model = Sequential()
model.add(ResNet50(input_shape=(224, 224, 3),weights="imagenet"
                             ,include_top=False))
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add( Dense(num_classes, activation='sigmoid',name='preds'))
model.layers[0].trainable= False
model.summary()

In [24]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

In [25]:
model.compile(
    # set the loss as binary_crossentropy
    loss=tf.keras.losses.binary_crossentropy,
    # set the optimizer as stochastic gradient descent
    optimizer=tf.keras.optimizers.SGD(lr=0.001),
    # set the metric as accuracy
    metrics=['accuracy']
)


# mock-train the model 
mod=model.fit(
    X_train[:,:,:,:],
    y_train[:],
    epochs=110,
    verbose=1,
    validation_data=(X_test[:,:,:,:], y_test[:]),
    callbacks=[es]
)


In [36]:
model.save("model_brain-2.h5")

In [27]:
# pretrained_cnn = tf.keras.models.load_model('./model_brain.h5')

# evaluate model on holdout set
# eval_score = pretrained_cnn.evaluate(X_test,y_test)
# # print loss score
# print('Eval loss:',eval_score[0])
# # print accuracy score
# print('Eval accuracy:',eval_score[1] )

In [30]:
predictions = (model.predict(X_test) > 0.5).astype("int32")

In [31]:
predictions

In [32]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test , predictions ) 

In [34]:
pd.DataFrame(mod.history)[['accuracy','val_accuracy']].plot()
plt.title("Accuracy")
plt.show()
pd.DataFrame(mod.history)[['loss','val_loss']].plot()
plt.title("Loss")
plt.show()